In [ ]:
%matplotlib widget

In [ ]:
from pathlib import Path
import numpy as np
import flammkuchen as fl
import json
from scipy.stats import pearsonr

from matplotlib import  pyplot as plt
import seaborn as sns
sns.set(style="ticks", palette="deep")
cols = sns.color_palette()
import ipywidgets as widgets

from lotr import LotrExperiment
from lavian_et_al_2025.data_location import master_motion

In [ ]:
master = master_motion / "LS" / "aHB"
fish_list = list(master.glob("*_f*"))
path = fish_list[0]

In [ ]:
# loading traces, reliability index, and sensory regressors
traces = fl.load(path / "filtered_traces.h5", "/detr")
exp = LotrExperiment(path)
rel_rois = fl.load(path / "reliable_rois.h5")['reliability_arr']
sensory_regressors = fl.load(path / "sensory_regressors.h5", "/regressors").values.T

In [ ]:
in_ahb = fl.load(path / "mask_and_idxs_ahb.h5")['cells_idx']

In [ ]:
# defining visually tuned neurons 
rel_rois = rel_rois[in_ahb]
selected_vis = np.where(rel_rois > 0.3)[0]

# loading HDNs
selected = fl.load(path / "selected.h5")


In [ ]:
####### find max correlation with visual motion for each neuron 

n_neurons = traces.shape[1]
n_directions = sensory_regressors.shape[0]

# Calculate correlation matrix
all_correlations = np.corrcoef(traces.T, sensory_regressors)

# Extract neuron-direction correlations (first n_neurons rows, last n_directions columns)
neuron_direction_corr = all_correlations[:n_neurons, n_neurons:]

# Find maximum absolute correlation for each neuron
max_corr = np.max(np.abs(neuron_direction_corr), axis=1)

In [ ]:
# max corr values for HDNs
max_corr_hdns = max_corr[selected]

# max corr values for all aHB neurons 
max_corr_rois = max_corr[in_ahb]

# max corr for visually tuned 
max_corr_vis = max_corr_rois[selected_vis]

In [ ]:
t_start_s = 50
t_end = exp.n_pts//2
t_lims = (t_start_s*exp.fn, t_end)
t_slice = slice(*t_lims)
cc_wnd = 3600
i_array = np.arange(t_slice.start, t_slice.stop, cc_wnd*exp.fn)
cc_mats = np.zeros((traces.shape[1], traces.shape[1], len(i_array)))

for n, i in enumerate(i_array):
    cc_mats[:, :, n] = np.corrcoef(traces[i:i + cc_wnd*exp.fn, :].T)
corr_mat = np.nanmean(cc_mats, 2)

In [ ]:
# negative correlation values for all neurons 
neg_corr = np.nanmin(corr_mat, axis=0)

neg_corr_hdns = neg_corr[selected]
neg_corr_ahb = neg_corr[in_ahb]
neg_corr_vis =  neg_corr_ahb[selected_vis]

In [ ]:
fig2, ax2 = plt.subplots(1, 2, figsize=(10,5), sharex=True)

ax2[0].hist(max_corr_vis, bins=50, color='skyblue', label='All ROIs')
ax2[0].hist(max_corr_hdns, bins=50, color='forestgreen', label='HDNs')
ax2[0].set_ylabel('# Neurons')
ax2[0].legend(prop={'size': 10})

ax2[0].spines['top'].set_visible(False)
ax2[0].spines['right'].set_visible(False)

ax2[1].scatter(max_corr_rois, neg_corr_ahb, color='skyblue', s=5, label='All ROIs')
ax2[1].scatter(max_corr_vis, neg_corr_vis, color='purple', s=5, label='Visually Tuned')
ax2[1].scatter(max_corr_hdns, neg_corr_hdns, color='forestgreen', s=5, label='HDNs')

ax2[1].legend(prop={'size': 10})

ax2[1].spines['top'].set_visible(False)
ax2[1].spines['right'].set_visible(False)

ax2[1].set_xlabel('Max correlation with visual motion')
ax2[0].set_xlabel('Max correlation with visual motion')
ax2[1].set_ylabel('Max negative correlation')
ax2[1].set_xlim([-0.2, 1])
ax2[0].set_xlim([-0.2, 1])
plt.subplots_adjust(left=0.1, bottom=0.2)

In [ ]:
fig3, ax3 = plt.subplots(1, 2, figsize=(5,2.5), sharex=True)

ax3[0].hist(max_corr_vis, bins=50, color='skyblue', label='All ROIs')
ax3[0].hist(max_corr_hdns, bins=50, color='forestgreen', label='HDNs')
ax3[0].set_ylabel('# Neurons')
ax3[0].legend(prop={'size': 10})

ax3[0].spines['top'].set_visible(False)
ax3[0].spines['right'].set_visible(False)

ax3[1].scatter(max_corr_vis, neg_corr_vis, color='purple', s=2, label='Visually Tuned')
ax3[1].scatter(max_corr_hdns, neg_corr_hdns, color='forestgreen', s=2, label='HDNs')

#ax3[1].legend(prop={'size': 10})
ax3[1].set_ylim([-1, -0.2])
ax3[1].set_xlim([0, 0.8])
ax3[0].set_xlim([0, 0.8])

ax3[1].spines['top'].set_visible(False)
ax3[1].spines['right'].set_visible(False)

ax3[1].set_xlabel('Max correlation with visual motion')
ax3[0].set_xlabel('Max correlation with visual motion')
ax3[1].set_ylabel('Max negative correlation')
plt.subplots_adjust(left=0.1, bottom=0.2, wspace=0.3, right=0.95)

In [ ]:
file_name = "max corr vs neg corr.pdf"
fig3.savefig(path / file_name, dpi=300)